In [2]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [3]:
# Define your connection parameters
connection_params = {
    'host': 'localhost', # e.g., 'localhost' or '127.0.0.1'
    'database': 'platformdb1', # Name of your database
    'user': 'root', # Your MySQL username
    'password': '0000', # Your MySQL password
}

df = pd.read_excel('output1.xlsx',engine= 'openpyxl')
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y')

In [7]:
try:
    # Establish the connection
    connection = mysql.connector.connect(**connection_params)
    
    if connection.is_connected():

        for table, column in [
            # ('dates', 'date'),
            # ('Brand', 'brand'),
            # ('Category', 'Category'),
            ('Reigon', 'Reigon'),
            ('SKU', 'sku'),
            ('SizeBracket','size_bracket')
        ]:
            cursor = connection.cursor()
            unique_values = df[column].drop_duplicates().dropna().tolist() 

            data_to_insert = [(v,) for v in unique_values]

            if column == 'date': 
                query = f"INSERT INTO {table} ({column}) VALUES (%s)"
            else : 
                query = f"INSERT INTO {table} ({column}_name) VALUES (%s)"

            cursor.executemany(query, data_to_insert)
            connection.commit()

            print("successfully ingested into the database.")
    

except Error as e:
    print("Error while connecting to MySQL", e)


In [ ]:
connection = mysql.connector.connect(**connection_params)

cursor = connection.cursor()

df['ID'] = range(1, len(df) + 1)

query = f"INSERT INTO Promotions (rawdata_id,Status) VALUES ({'%s, %s'})"

df['Promoted'].fillna(False, inplace=True) 
data_to_insert = df[['ID', 'Promoted']].values.tolist()


cursor.executemany(query, data_to_insert)

# Commit the transaction
connection.commit()


connection.close()

In [10]:
connection = mysql.connector.connect(**connection_params)

cursor = connection.cursor()
if connection.is_connected():

    category_id = size_bracket_id = sku_id = None

    # Now, insert data into the fact_sales table
    for index, row in df.iterrows():
        # Fetch foreign keys based on the unique names
        cursor.execute("SELECT date_id FROM dates WHERE date = (%s)", (row['date'],))
        date_id = cursor.fetchone()[0]
        
        cursor.execute("SELECT brand_id FROM Brand WHERE brand_name = %s", (row['brand'],))
        brand_id = cursor.fetchone()[0]
        
        if pd.notna(row['Category']) :
            cursor.execute("SELECT category_id FROM Category WHERE category_name = %s", (row['Category'],))
            category_id = cursor.fetchone()[0]
        
        if pd.notna(row['size_bracket']) :
            cursor.execute("SELECT size_bracket_id FROM sizebracket WHERE size_bracket_name = %s", (row['size_bracket'],))
            size_bracket_id = cursor.fetchone()[0]
        
        cursor.execute("SELECT Reigon_id FROM Reigon WHERE Reigon_name = %s", (row['Reigon'],))
        reigon_id = cursor.fetchone()[0]

        if pd.notna(row['sku']) :
            cursor.execute("SELECT sku_id FROM SKU WHERE sku_name = %s", (row['sku'],))
            sku_id = cursor.fetchone()[0]
            
         # Prepare the dynamic SQL query and parameters
        columns = ['date_id', 'brand_id', 'Reigon_id']
        values = [date_id, brand_id, reigon_id]
        
        if category_id is not None:
            columns.append('category_id')
            values.append(category_id)
        
        if size_bracket_id is not None:
            columns.append('size_bracket_id')
            values.append(size_bracket_id)
        
        if sku_id is not None:
            columns.append('sku_id')
            values.append(sku_id)

        if pd.notna(row['Sales (KGS) (000)']):
            columns.append('volume_sales')
            values.append(row['Sales (KGS) (000)'])

        if pd.notna(row['Sales (KGS) (000)']):
            columns.append('value_sales')
            values.append(row['Sales Value (000)'])
        
        if pd.notna(row['Weighted Distribution - Reach']):
            columns.append('WOD')
            values.append(row['Weighted Distribution - Reach'])

        # Build the final query
        insert_query = f"INSERT INTO RawData ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"
        
        # Execute the dynamic query
        cursor.execute(insert_query, tuple(values))

    # Commit the transaction
    connection.commit()
    print("Data inserted successfully into RawData table.")

(1, 2, 1, 1, 1, 1)
(1, 2, 1, 1, 1, 1, 0.01, 0.572, 0.42)
(1, 2, 1, 2, 1, 2, 0.005, 0.235, 0.204)
(1, 3, 1, 2, 2, 3)
(1, 3, 1, 2, 2, 4, 0.225, 2.446, 0.89)
(1, 3, 1, 2, 2, 5, 0.031, 0.239, 0.06)
(1, 3, 1, 2, 2, 6, 62.92, 815.6, 70.277)
(1, 3, 1, 2, 2, 7, 64.939, 709.405, 45.853)
(1, 3, 1, 2, 2, 8)
(1, 3, 1, 2, 2, 9, 0.009, 0.116, 0.073)
(1, 3, 1, 2, 2, 10)
(1, 3, 1, 2, 2, 11)
(1, 3, 1, 2, 2, 12)
(1, 3, 1, 2, 2, 13, 1.432, 15.418, 0.822)
(1, 3, 1, 2, 2, 14)
(1, 3, 1, 2, 2, 15)
(1, 3, 1, 2, 2, 16)
(1, 3, 1, 2, 3, 17, 32.365, 354.79, 18.965)
(1, 3, 1, 2, 3, 18, 0.012, 0.178, 0.019)
(1, 3, 1, 2, 1, 19, 0.047, 0.0)
(1, 3, 1, 2, 1, 20)
(1, 3, 1, 2, 1, 21)
(1, 3, 1, 2, 1, 22)
(1, 4, 1, 3, 4, 23, 0.208, 23.953, 13.353)
(1, 4, 1, 3, 4, 24, 0.978, 19.857, 16.52)
(1, 4, 1, 3, 4, 25, 2.067, 46.149, 0.729)
(1, 4, 1, 3, 4, 26, 11.654, 393.812, 40.754)
(1, 4, 1, 3, 4, 27)
(1, 4, 1, 3, 4, 28, 31.43, 462.086, 18.639)
(1, 4, 1, 3, 4, 29, 0.014, 0.511, 0.463)
(1, 4, 1, 3, 4, 30)
(1, 4, 1, 3, 4, 31)
(1, 4,

KeyboardInterrupt: 